In [1]:
import pandas as pd

In [2]:
interactions = pd.read_csv("../Preprocessing/processed_dataframes/interactions.csv")
recipes = pd.read_csv("../Preprocessing/processed_dataframes/recipes_with_text.csv")
recipes.sort_values(by=["recipe_id"], inplace=True)

Average ratings per recipe

In [3]:
avg_ratings = interactions.drop(["user_id"], axis=1).groupby("recipe_id", as_index=False).mean().sort_values(by=["rating"], ascending=False)

Show Top 20 most rated recipes

In [4]:
for _, row in avg_ratings.head(50).iterrows():
    recipe_data = recipes.iloc[int(row["recipe_id"])]
    
    print("Name:", recipe_data["name"], f"({int(row['recipe_id'])})")
    print()
    print(", ".join(eval(recipe_data["steps"])))
    print("-"*50)

Name: roast chicken with grand marnier glaze (112390)

prep the chicken, remove the insides and rinse with water and then pat dry, rub the bird with olive oil and then season with salt , pepper , and rosemary, place in roasting pan and put the orange and onion quarters inside, roast at 400 degrees for 20 minutes, then turn your oven down to 350 degrees for 1 hour more, while bird is cooking , make your glaze, melt the butter , mustard , honey , and apricot jam til bubbly and then take off heat and add grand marnier, can keep on low to keep warm, when there is 40 minutes left of cook time , pour about half the glaze over the bird and then let it finish cooking, pull bird out , let rest for 10 minutes so juices can recollect, after carving it up and putting meat on platter , drizzle the pieces with the rest of the glaze, garnish with fresh rosemary sprigs
--------------------------------------------------
Name: forty four (134173)

get a large , clean glass jar that you can tightly close

## Extend training data

In [4]:
train_csv = pd.read_csv("../Preprocessing/processed_dataframes/train.csv")
my_user_id = interactions["user_id"].max() + 1
my_ratings = pd.DataFrame.from_dict({
    "user_id": [my_user_id]*16,
    "recipe_id": [134161, 134170, 134171, 134191, 134159, 134152, 134151, 134128, 134235, 134257, 134138, 134139, 134189, 134192, 134153, 134193],
    "rating": [2, 5, 3, 4, 2, 1, 4, 1, 3, 5, 5, 3, 1, 1, 1, 1]
})
# my_ratings.to_csv("my_ratings.csv", index=False)

extended_train = pd.concat([train_csv, my_ratings])
# extended_train.to_csv("extended_train.csv", index=False)

## Train on extended data

In [6]:
import sys
sys.path.append("../")

from torch.utils.data import DataLoader

n_users = 226570+1
n_items = 231637

In [7]:
batch_size = 4086

train_path = "extended_train.csv"
val_path = "../Preprocessing/processed_dataframes/val.csv"

def get_dataloader(csv_path, dataset, has_rating_column, batch_size, num_workers, shuffle, **kwargs):
    return DataLoader(
        dataset(
            interactions_file=csv_path,
            n_users=n_users,
            n_items=n_items,
            has_rating_column=has_rating_column,
            **kwargs,
        ), 
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=shuffle,
    )


from DataLoader.simple_dataset import SimpleFoodRatingsDataset

simple_train_dataloader = get_dataloader(train_path, SimpleFoodRatingsDataset, True, batch_size, 8, True)
simple_val_dataloader = get_dataloader(val_path, SimpleFoodRatingsDataset, True, batch_size, 2, True)

from Training.simple.simple_train_functions import fit, eval_test

losses = fit(
    simple_train_dataloader, 
    simple_val_dataloader,
    n_users,
    n_items,
    k_gmf=8,
    k_mlp=8,
    layer_sizes=[16, 32, 16, 8],
    alpha=0.5,
    lr=0.001,
    epochs=5,
    weight_path="simple_neumf",
    run_number=2,
    random_state=None,
    show_loss=True,
)

  0%|          | 0/5 [00:00<?, ?it/s]

epoch: 1, train_loss: 3.8528729576087293, val_loss: 1.1323010689336248
epoch: 2, train_loss: 0.9301545356205536, val_loss: 0.9101262817584425
epoch: 3, train_loss: 0.8598107795721024, val_loss: 0.9040594186719084
epoch: 4, train_loss: 0.8195505210274572, val_loss: 0.9134523987622389
epoch: 5, train_loss: 0.7640598005822008, val_loss: 0.9324634327996955


In [9]:
import os
from Training.simple.simple_train_functions import define_model
import torch

trained_model = define_model(n_users, n_items, k_gmf=8, k_mlp=8, layer_sizes=[16, 32, 16, 8], alpha=0.5)
trained_model.load_state_dict(torch.load(f"simple_neumf/run_2/3-0.9040594186719084.pt"))

<All keys matched successfully>

Get items ids that are not rated by the user

In [10]:
unrated_ids = []
for i in range(n_items):
    if i not in my_ratings["recipe_id"].tolist():
        unrated_ids.append(i)

In [11]:
with torch.no_grad():
    predictions_for_me = trained_model(
        torch.tensor([my_user_id]*len(unrated_ids)).cuda(), 
        torch.tensor(unrated_ids).cuda()
    ).ravel().cpu().numpy()
    print(predictions_for_me.shape)

(231621,)


Top 10 recommended recipes

In [12]:
id_rating_pair_list = [(prediction, recipe_id) for recipe_id, prediction in zip(unrated_ids, predictions_for_me)]
id_rating_pair_list.sort(reverse=True)

In [16]:
for _, recipe_id in id_rating_pair_list[:10]:
    recipe_data = recipes.iloc[recipe_id]
#     print(recipe_data["name"])
    print("Name:", recipe_data["name"])
    print()
    print(", ".join(eval(recipe_data["steps"])))
    print("-"*50)

Name: just peachy grillin  ribs  rsc

prepare a gas grill for direct grilling over medium heat, on a work surface , remove the membrane from the back of the ribs, in a small bowl , stir together kosher salt , rosemary , sage , and granulated garlic until mixed, generously season both sides of ribs with spice mixture, wrap each seasoned pork slab in reynolds wrap foil, place the foil-wrapped ribs on the grill in a single layer, cover the grill, grill , turning packets once halfway through cooking , about 1- 1 1 / 2 hours or until meat is tender and pulls away from the bones, meanwhile , make the sauce, in a food processor , add the peaches, process until slightly chunky, transfer peaches to a medium bowl, add barbecue sauce , mustard , honey , and lemon juice, stir until well combined, when the ribs are cooked , open the packets, brush both sides of ribs with sauce, place ribs on grill rack, grill , turning once and brushing with additional sauce , 3-4 minutes per side, transfer ribs to